In [ ]:
import time
import tracemalloc
import functools
import threading
import psutil
import os
import random

In [ ]:
def timeit(func):
    """
    Декоратор для измерения времени работы функции.
    Суммирует время всех рекурсивных вызовов (если функция сама себя вызывает).
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        if not hasattr(wrapper, "_time_acc"):
            wrapper._time_acc = 0.0
        start = time.perf_counter()
        try:
            return func(*args, **kwargs)
        finally:
            end = time.perf_counter()
            wrapper._time_acc += (end - start)
            # если это внешний вызов (не рекурсивный), то печатаем и сбрасываем
            # мы определим «внешний вызов» как момент, когда стек глубже, чем вызов функции
            wrapper._recursion_depth -= 1
            if wrapper._recursion_depth == 0:
                total = wrapper._time_acc
                # сброс для следующего полного запуска
                wrapper._time_acc = 0.0
                print(f"[timeit] Function {func.__name__} took total {total:.6f} seconds")
        # Для корректного определения рекурсии:

    def new_wrapper(*args, **kwargs):
        if not hasattr(wrapper, "_recursion_depth"):
            wrapper._recursion_depth = 0
        wrapper._recursion_depth += 1
        return wrapper(*args, **kwargs)

    return new_wrapper


def memoryit(func):
    """
    Декоратор для измерения пикового использования памяти Python во время работы функции.
    Использует tracemalloc для Python-памяти + psutil для общего RSS процесса.
    Аккумулирует пиковые значения в рекурсии.
    """
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # инициализация
        if not hasattr(wrapper, "_mem_peak_python"):
            wrapper._mem_peak_python = 0
            wrapper._mem_peak_rss = 0
        if not hasattr(wrapper, "_call_depth"):
            wrapper._call_depth = 0

        if wrapper._call_depth == 0:
            # первый (внешний) вызов — запускаем трассировку
            tracemalloc.start()
        wrapper._call_depth += 1

        rss_before = psutil.Process(os.getpid()).memory_info().rss

        try:
            return func(*args, **kwargs)
        finally:
            current, peak = tracemalloc.get_traced_memory()
            rss_after = psutil.Process(os.getpid()).memory_info().rss

            # обновляем пик Python-аллокатора
            if peak > wrapper._mem_peak_python:
                wrapper._mem_peak_python = peak
            # обновляем пик RSS
            rss_delta = rss_after - rss_before
            if rss_delta > wrapper._mem_peak_rss:
                wrapper._mem_peak_rss = rss_delta

            wrapper._call_depth -= 1
            if wrapper._call_depth == 0:
                # завершение внешнего вызова
                tracemalloc.stop()
                # вывод результатов
                print(
                    f"[memoryit] Function {func.__name__} peak python allocation: {wrapper._mem_peak_python / 1024:.2f} KiB")
                print(f"[memoryit] Function {func.__name__} peak RSS delta: {wrapper._mem_peak_rss / 1024:.2f} KiB")
                # сброс для следующего вызова
                wrapper._mem_peak_python = 0
                wrapper._mem_peak_rss = 0

    return wrapper

In [ ]:
def _selection_sort(items: list[int]) -> None:
    n = len(items)
    for i in range(n):
        min_idx = i
        for j in range(i + 1, n):
            if items[j] < items[min_idx]:  # ищем самый маленький
                min_idx = j
        items[i], items[min_idx] = items[min_idx], items[i]


def _join_numbers(items: list[int]) -> str:
    if not items:
        return ""
    result = str(items[0])
    for value in items[1:]:
        result += " " + str(value)
    return result


@timeit
@memoryit
def symmetric_difference(input_str: str) -> str:
    set_a: list[int] = []
    set_b: list[int] = []
    result: list[int] = []
    input_list = input_str.split()

    is_b = False

    for symbol in input_list:
        number = int(symbol)
        if number == 0:
            if is_b:
                break
            is_b = True
            continue
        if not is_b:
            set_a.append(number)
        else:
            set_b.append(number)

    for value in set_a:
        if value not in set_b:
            result.append(value)

    for value in set_b:
        if value not in set_a:
            result.append(value)

    _selection_sort(result)

    if not result:
        return "0"
    return _join_numbers(result)


print(symmetric_difference('1500 7800 12345 567 8900 4321 9999 150 6789 3210 2555 7777 11111 5432 8888 6666 4444 2222 3333 5555 0 7800 567 8900 20000 150 123 4567 9999 4321 6789 3210 11111 8888 6666 4444 0'))
print(symmetric_difference('100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 0 50 150 250 350 450 550 650 750 850 950 1050 1150 1250 1350 1450 1550 1650 1750 1850 1950 2050 2150 2250 2350 2450 0'))
print(symmetric_difference('42 1984 2023 777 1337 69 420 1234 5678 9999 11111 15555 17777 19999 2468 13579 8642 7531 2222 3333 4444 5555 6666 7777 8888 0 42 777 1337 10000 15000 20000 2468 8642 2222 4444 6666 8888 11111 15555 17777 19999 0'))
print(symmetric_difference('1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 0 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 0'))
print(symmetric_difference('15000 2500 7500 12500 500 17500 1000 20000 3000 4000 6000 7000 8000 9000 11000 12000 13000 14000 16000 17000 18000 19000 0 2500 7500 12500 17500 333 666 999 1333 1666 1999 2333 2666 2999 3333 3666 3999 4333 4666 4999 0'))
print(symmetric_difference('111 222 333 444 555 666 777 888 999 1111 2222 3333 4444 5555 6666 7777 8888 9999 11111 12222 13333 14444 15555 16666 17777 18888 19999 0 222 444 666 888 1111 3333 5555 7777 9999 12222 14444 16666 18888 20000 12345 13579 14789 15987 0'))
print(symmetric_difference('123 456 789 1011 1213 1415 1617 1819 2021 2223 2425 2627 2829 3031 3233 3435 3637 3839 4041 4243 4445 4647 4849 5051 5253 5455 5657 5859 6061 6263 0 456 789 1011 1415 1819 2223 2627 3031 3435 3839 4243 4647 5051 5455 5859 6263 6666 7777 8888 9999 11111 0'))
print(symmetric_difference('5000 10000 15000 2000 4000 6000 8000 12000 14000 16000 18000 2500 3500 4500 5500 6500 7500 8500 9500 10500 11500 12500 13500 14500 15500 16500 17500 18500 19500 0 2000 4000 6000 8000 10000 12000 14000 16000 18000 2500 4500 6500 8500 10500 12500 14500 16500 18500 19500 333 666 999 0'))
print(symmetric_difference('11111 12222 13333 14444 15555 16666 17777 18888 19999 20000 1111 2222 3333 4444 5555 6666 7777 8888 9999 1234 2345 3456 4567 5678 6789 7890 8901 9012 1357 2468 3579 4680 5791 6802 7913 8024 9135 0 12222 14444 16666 18888 20000 2222 4444 6666 8888 2345 4567 6789 8901 2468 4680 6802 8024 10000 15000 0'))
print(symmetric_difference('19999 20000 1 2 3 4 5 6 7 8 9 10 100 200 300 400 500 600 700 800 900 1000 1111 2222 3333 4444 5555 6666 7777 8888 9999 12345 13579 14999 15999 16999 17999 18999 0 1 3 5 7 9 100 300 500 700 900 1111 3333 5555 7777 9999 12345 14999 15999 17999 18999 20000 42 142 242 342 442 0'))

In [ ]:
@timeit
@memoryit
def long_a_s(first_operand: str, operator: str, second_operand: str) -> str:
    left = int(first_operand)
    right = int(second_operand)
    if operator == "+":
        return str(left + right)
    if operator == "-":
        return str(left - right)


print(long_a_s("123", "+", "999"))
print(long_a_s("1000", "-", "1"))
print(long_a_s("-50", "+", "25"))
print(long_a_s("500", "-", "1234"))


In [ ]:
def solve_magaraja(N, K):
    if K == 0:
        return 1
    if K == 1:
        return N * N

    cells = [(i, j) for i in range(N) for j in range(N)]
    total = len(cells)
    count = 0

    def attacks(a, b):
        r1, c1 = a
        r2, c2 = b
        if r1 == r2 or c1 == c2 or abs(r1 - r2) == abs(c1 - c2):
            return True
        dr = abs(r1 - r2)
        dc = abs(c1 - c2)
        if (dc == 2 and dr == 1) or (dc == 1 and dr == 2):
            return True
        return False

    def back_track(idx, placed):
        nonlocal count
        if len(placed) == K:
            count += 1
            return
        if idx >= total:
            return
        if len(placed) + (total - idx) < K:
            return
        back_track(idx + 1, placed)
        candidate = cells[idx]
        safe = True
        for p in placed:
            if attacks(candidate, p):
                safe = False
                break
        if safe:
            placed.append(candidate)
            back_track(idx + 1, placed)
            placed.pop()

    back_track(0, [])
    return count

def magaraja():
    data = input().split()
    if not data:
        return
    N = int(data[0])
    K = int(data[1])
    result = solve_magaraja(N, K)
    print(result)

# ТЕСТЫ
print("=== ТЕСТИРОВАНИЕ МАГАРАДЖИ ===")

# Тест 1: минимальная доска
print("\nТест 1: N=1, K=0")
print("Ожидаем: 1")
print("Получаем:", solve_magaraja(1, 0))

# Тест 2: одна фигура на минимальной доске
print("\nТест 2: N=1, K=1")
print("Ожидаем: 1")
print("Получаем:", solve_magaraja(1, 1))

# Тест 3: слишком много фигур
print("\nТест 3: N=1, K=2")
print("Ожидаем: 0")
print("Получаем:", solve_magaraja(1, 2))

# Тест 4: маленькая доска 2x2
print("\nТест 4: N=2, K=1")
print("Ожидаем: 4")
print("Получаем:", solve_magaraja(2, 1))

print("\nТест 5: N=2, K=2")
print("Ожидаем: 0 (все фигуры атакуют друг друга)")
print("Получаем:", solve_magaraja(2, 2))

# Тест 5: доска 3x3
print("\nТест 6: N=3, K=1")
print("Ожидаем: 9")
print("Получаем:", solve_magaraja(3, 1))

print("\nТест 7: N=3, K=2")
print("Ожидаем: >0")
result = solve_magaraja(3, 2)
print("Получаем:", result)
print("Статус:", "✓ Есть решения" if result > 0 else "✗ Нет решений")

# Тест 6: доска 4x4
print("\nТест 8: N=4, K=2")
print("Ожидаем: >0")
result = solve_magaraja(4, 2)
print("Получаем:", result)
print("Статус:", "✓ Есть решения" if result > 0 else "✗ Нет решений")

# Тест 7: граничный случай
print("\nТест 9: N=0, K=0")
print("Ожидаем: 1")
print("Получаем:", solve_magaraja(0, 0))

print("\n=== ТЕСТИРОВАНИЕ ЗАВЕРШЕНО ===")

# Демонстрация работы оригинальной функции
print("\n=== ДЕМОНСТРАЦИЯ РАБОТЫ ОРИГИНАЛЬНОЙ ФУНКЦИИ ===")
print("Введите тест вручную (N K):")
magaraja()

In [ ]:
@timeit
@memoryit
def broken_search(target: int, nums: str) -> int:
    target = str(target)

    try:
        return nums.index(target)
    except ValueError:
        return -1


numbers = [str(random.randint(0, 200000)) for _ in range(10000)]
nums_str = " ".join(numbers)
print(broken_search(random.randint, nums_str))

[memoryit] Function broken_search peak python allocation: 1.35 KiB
[memoryit] Function broken_search peak RSS delta: 0.00 KiB
[timeit] Function broken_search took total 0.000403 seconds
-1


In [ ]:
@timeit
@memoryit
def _selection_sort(items: list, priorities: list) -> None:
    n = len(items)
    for i in range(n):
        min_idx = i
        for j in range(i + 1, n):
            need_swap = False
            for p in priorities:
                idx = int(p) - 1
                if items[j][1][idx] > items[min_idx][1][idx]:
                    need_swap = True
                    break
                elif items[j][1][idx] < items[min_idx][1][idx]:
                    break
            if need_swap:
                min_idx = j
        items[i], items[min_idx] = items[min_idx], items[i]

test_data = [
    (["3", "3", "2 1 3", "A 1 2 3", "B 3 2 1", "C 3 1 2"], ["B", "A", "C"]),
    (["3", "3", "1 2 3", "X 5 1 1", "Y 3 9 9", "Z 4 0 0"], ["X", "Z", "Y"]),
    (["3", "1", "1", "One 10", "Two 30", "Three 20"], ["Two", "Three", "One"])
]

print("=== ТЕСТЫ ===")
for i, (inputs, expected) in enumerate(test_data, 1):
    n = int(inputs[0])
    k = int(inputs[1])
    priorities = inputs[2].split()

    records = []
    for j in range(3, 3 + n):
        data = inputs[j].split()
        name = data[0]
        values = [int(x) for x in data[1:]]
        records.append((name, values))

    _selection_sort(records, priorities)
    result = [name for name, _ in records]

    status = "✅ ПРОЙДЕН" if result == expected else "❌ НЕ ПРОЙДЕН"
    print(f"Тест {i}: {status}")
    print(f"Ожидалось: {expected}")
    print(f"Получилось: {result}\n")

In [ ]:
@timeit
@memoryit
def very_fast_sort(N, K, M, L):

    if L == 0:  # если L = 0, все элементы будут 0
        print(0)
        return

    count = [0] * L  # массив для подсчета количества каждого элемента

    current = K  # генерируем последовательность и считаем элементы
    count[current % L] += 1  # генерируем последовательность и считаем элементы

    for i in range(1, N):
        current = (current * M) % (4294967295) % L
        count[current] += 1

    # собираем отсортированную последовательность и считаем сумму нечетных позиций
    result = 0
    pos = 1  # текущая позиция в отсортированном массиве

    for num in range(L):
        for j in range(count[num]):
            if pos % 2 == 1:  # нечетная позиция
                result = (result + num) % L
            pos += 1

    print(result)


print("Тест 1:", very_fast_sort(5, 7, 13, 100), "(ожидается 77)")
print("Тест 2:", very_fast_sort(3, 5, 1, 10), "(ожидается 0)")
print("Тест 3:", very_fast_sort(1, 42, 13, 100), "(ожидается 42)")
print("Тест 4:", very_fast_sort(3, 1, 2, 10), "(ожидается 5)")
print("Тест 5:", very_fast_sort(4, 0, 5, 10), "(ожидается 0)")

In [ ]:
@timeit
@memoryit
def run_solution(N, M, matrix):
    a = [row[:] for row in matrix]  # копируем матрицу
    ops = []

    # восстановление строк
    for i in range(N):
        need_min = i * M + 1
        found_pos = 0

        for r in range(i, N):
            current_min = a[r][0]
            for val in a[r]:
                if val < current_min:
                    current_min = val
            if current_min == need_min:
                found_pos = r
                break

        if found_pos != i and found_pos != -1:
            ops.append(f"R {i + 1} {found_pos + 1}")
            temp = a[i]
            a[i] = a[found_pos]
            a[found_pos] = temp

    # восстановление столбцов
    for j in range(M):
        need_val = j + 1
        found_pos = 0

        for c in range(j, M):
            if a[0][c] == need_val:
                found_pos = c
                break

        if found_pos != j and found_pos != -1:
            ops.append(f"C {j + 1} {found_pos + 1}")
            for r in range(N):
                temp = a[r][j]
                a[r][j] = a[r][found_pos]
                a[r][found_pos] = temp

    return ops

@timeit
@memoryit
def apply_operations(N, M, matrix, ops):
    # gрименяет операции к матрице и возвращает результат
    a = [row[:] for row in matrix]

    for op in ops:
        parts = op.split()
        type_op = parts[0]
        x = int(parts[1]) - 1
        y = int(parts[2]) - 1

        if type_op == 'R':
            # Меняем строки
            temp = a[x]
            a[x] = a[y]
            a[y] = temp
        else:  # 'C'
            # Меняем столбцы
            for r in range(N):
                temp = a[r][x]
                a[r][x] = a[r][y]
                a[r][y] = temp

    return a

@timeit
@memoryit
def is_correct_final_matrix(N, M, matrix):
    # является ли матрица исходной
    for i in range(N):
        for j in range(M):
            if matrix[i][j] != i * M + j + 1:
                return False
    return True


def print_matrix(matrix):
    for row in matrix:
        print(' '.join(map(str, row)))


def run_test(test_name, N, M, input_matrix):
    print(f"\n=== {test_name} ===")
    print("Входная матрица:")
    print_matrix(input_matrix)

    ops = run_solution(N, M, input_matrix)

    print(f"\nОперации ({len(ops)}):")
    for op in ops:
        print(op)

    # Проверяем корректность
    result_matrix = apply_operations(N, M, input_matrix, ops)
    print("\nРезультирующая матрица:")
    print_matrix(result_matrix)

    is_correct = is_correct_final_matrix(N, M, result_matrix)
    print(f"\nКорректность: {is_correct}")

    return is_correct, ops


# Тест 1: Пример из условия
def test1():
    N, M = 2, 2
    matrix = [
        [4, 3],
        [2, 1]
    ]
    return run_test("Тест 1 (пример из условия)", N, M, matrix)


# Тест 2: Уже правильная матрица
def test2():
    N, M = 3, 5
    matrix = [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [11, 12, 13, 14, 15]
    ]
    return run_test("Тест 2 (уже правильная)", N, M, matrix)


# Тест 3: Пример из условия
def test3():
    N, M = 4, 5
    matrix = [
        [10, 7, 9, 8, 6],
        [15, 12, 14, 13, 11],
        [20, 17, 19, 18, 16],
        [5, 2, 4, 3, 1]
    ]
    return run_test("Тест 3 (пример из условия)", N, M, matrix)


# Тест 4: Только строки перепутаны
def test4():
    N, M = 3, 3
    matrix = [
        [7, 8, 9],
        [1, 2, 3],
        [4, 5, 6]
    ]
    return run_test("Тест 4 (только строки)", N, M, matrix)


# Тест 5: Только столбцы перепутаны
def test5():
    N, M = 2, 4
    matrix = [
        [1, 4, 3, 2],
        [5, 8, 7, 6]
    ]
    return run_test("Тест 5 (только столбцы)", N, M, matrix)


# Тест 6: Матрица 1x1
def test6():
    N, M = 1, 1
    matrix = [[1]]
    return run_test("Тест 6 (1x1)", N, M, matrix)


# Тест 7: Случайная перестановка
def test7():
    N, M = 2, 3
    matrix = [
        [4, 5, 6],
        [1, 2, 3]
    ]
    return run_test("Тест 7 (случайная)", N, M, matrix)


def run_all_tests():
    """Запускает все тесты"""
    tests = [test1, test2, test3, test4, test5, test6, test7]
    passed = 0

    print("=" * 50)
    print("ЗАПУСК АВТОТЕСТОВ")
    print("=" * 50)

    for test in tests:
        try:
            correct, ops = test()
            if correct:
                passed += 1
                print("✅ ТЕСТ ПРОЙДЕН")
            else:
                print("❌ ТЕСТ НЕ ПРОЙДЕН")
        except Exception as e:
            print(f"❌ ОШИБКА В ТЕСТЕ: {e}")

    print("\n" + "=" * 50)
    print(f"ИТОГ: {passed}/{len(tests)} тестов пройдено")
    print("=" * 50)

run_all_tests()

In [ ]:
def _is_better_result(left: tuple[str, int, int], right: tuple[str, int, int]) -> bool:
    if left[1] != right[1]:  #  больше задач — выше
        return left[1] > right[1]
    if left[2] != right[2]:  #  при равенстве сравниваем
        return left[2] < right[2]
    return left[0] < right[0]  # дальше идёт алф порядок


def _sift_down(heap: list[tuple[str, int, int]], start: int, size: int) -> None:
    parent = start
    while True:
        left = 2 * parent + 1  # левый потомок
        right = left + 1       # правый потомок
        best = parent
        
        # сравниваем с левым потомком
        if left < size and _is_better_result(heap[left], heap[best]):
            best = left
        # сравниваем с правым потомком  
        if right < size and _is_better_result(heap[right], heap[best]):
            best = right
        
        # если родитель лучше обоих потомков - выходим
        if best == parent:
            break
            
        # меняем родителя с лучшим потомком
        heap[parent], heap[best] = heap[best], heap[parent]
        parent = best


@timeit
@memoryit
def heap_sort_results(participants: list[tuple[str, int, int]]) -> list[str]:
    heap = participants[:]  # копия входа
    size = len(heap)
    
    # меняем кучи
    for idx in range(size // 2, -1, -1):
        _sift_down(heap, idx, size)
    
    result: list[str] = []
    
    # сортировка
    for end in range(size - 1, -1, -1):
        # перемещаем лучший элемент в конец
        heap[0], heap[end] = heap[end], heap[0]
        result.append(heap[end][0])  # забираем имя участника
        _sift_down(heap, 0, end)    # восстанавливаем кучу
    return result


contestants = [
    ("alla", 4, 100),
    ("gena", 6, 1000),
    ("gosha", 2, 90),
    ("rita", 2, 90),
    ("timofey", 4, 80),
]
print(heap_sort_results(contestants))
print(heap_sort_results([("aaa", 1, 5), ("bbb", 1, 5), ("ccc", 2, 7)]))
print(heap_sort_results([("x", 0, 0), ("y", 0, 0)]))

In [ ]:
@timeit
@memoryit
def is_heap_property(values: list[int]) -> str:
    n = len(values)
    for idx in range(n):
        left = 2 * idx + 1  # левый child в массиве
        right = left + 1  # правый child
        if left < n and values[idx] > values[left]:
            return "NO"
        if right < n and values[idx] > values[right]:
            return "NO"
    return "YES"


print(is_heap_property([1, 3, 5, 7, 9]))
print(is_heap_property([5, 4, 3, 2, 1]))
print(is_heap_property([2]))
print(is_heap_property([2, 2, 2, 2]))

In [ ]:
@timeit
@memoryit
def nested_segments(segments: list[tuple[int, int]], points: list[int]) -> list[int]:
    if not segments or not points:
        return [-1] * len(points)

    answers = [-1] * len(points)
    stack: list[tuple[int, int]] = []  # (конец отрезка, индекс отрезка)
    seg_idx = 0  # seg_idx - указатель на текущий отрезок
    total_segments = len(segments)

    for point_idx, point in enumerate(points):
        #  добавляем все отрезки начинающиеся до текущей точки
        while seg_idx < total_segments and segments[seg_idx][0] <= point:
            start, end = segments[seg_idx]
            while stack and stack[-1][0] < start:
                stack.pop()
            stack.append((end, seg_idx + 1))
            seg_idx += 1

        # удаляем отрезки, которые не содержат текущую точку
        while stack and stack[-1][0] < point:
            stack.pop()

        # записываем результат
        if stack:
            answers[point_idx] = stack[-1][1]
        else:
            answers[point_idx] = -1
    return answers


test_segments = [(1, 10), (2, 5), (6, 9)]
test_points = [1, 3, 6, 11]
print(nested_segments(test_segments, test_points))
print(nested_segments([(1, 4), (2, 3)], [0, 2, 4]))
print(nested_segments([(1, 5), (5, 10)], [5, 6, 1]))



In [ ]:
# TODO добавить тексты задач, проверить тесты